In [1]:
import pandas as pd
import numpy as np

# Базовая модель

Модель должна считать частоту отправки 

Загружаем данные

In [2]:
source_data = pd.read_csv('./data/tink.csv')

In [3]:
source_data.drop(columns=source_data.columns[0], axis=1, inplace=True)
source_data['date'] = pd.to_datetime(source_data.date)
source_data

,payer,date,amount,phone
0,256082,2020-09-25,100,79217988811
1,577125,2020-09-25,100,79211031739
2,332492,2020-09-25,2000,79220885615
3,1093977,2020-09-25,100,79215570961
4,1037953,2020-09-25,10000,79221073575
...,...,...,...,...
64115073,421252,2021-03-23,1000,79220416876
64115074,765025,2021-03-23,5000,79220751544
64115075,230665,2021-03-23,4000,79216171959
64115076,115111,2021-03-23,500,79215595398


Группируем по получателю, телефону и дате

In [4]:
pivot_data = pd.pivot_table(source_data, index=['payer','phone','date'],values=['amount'],aggfunc=np.sum)
pivot_data.sort_values(by=['payer', 'date'], inplace=True)
pivot_data

amount
payer   phone       date              
0       79216341986 2020-09-25     500
        79213906883 2020-09-30   18000
        79213903935 2020-12-12     100
1       79213033859 2020-09-25    5000
        79213029046 2020-09-28   11000
...                                ...
1310927 79220081626 2021-03-11    1500
                    2021-03-13    3000
        79219199310 2021-03-14    1500
        79219503515 2021-03-17     100
1310928 79210024786 2020-09-26     500

[58184279 rows x 1 columns]

Получаем последнего человека, которому каждый отправитель скидывал деньги

In [5]:
last_person_data = pivot_data.groupby(level=0).tail(1)
last_person_data

,,,amount
payer,phone,date,
0,79213903935,2020-12-12,100
1,79215518554,2021-03-19,59500
2,79216782855,2021-03-15,500
3,79215927097,2021-03-21,5000
4,79217427559,2021-03-21,1500
...,...,...,...
1310924,79214836154,2021-03-14,1600
1310925,79212463953,2021-03-16,100
1310926,79215258174,2021-03-22,1000


Получаем данные без последнего человека и дропаем дату

In [8]:
X_data = pivot_data[~pivot_data.isin(last_person_data)].dropna(axis=0)
X_data = X_data.groupby(by=['payer', 'phone']).sum()
X_data

Получаем частотность

In [9]:
count_data = pd.pivot_table(source_data, index=['payer', 'phone'],values=['amount'],aggfunc='size').rename('count')
count_data

payer    phone      
0        79213903935    1
         79213906883    1
         79216341986    1
1        79212770861    1
         79212968340    1
                       ..
1310927  79220208944    1
         79220350657    1
         79220609850    1
         79221098683    1
1310928  79210024786    1
Name: count, Length: 22335771, dtype: int64

In [13]:
X_data['count'] = count_data
X_data.sort_values(by=['payer', 'count'], inplace=True)
X_data

amount  count
payer   phone                       
0       79213906883   18000.0      1
        79216341986     500.0      1
1       79212770861    2000.0      1
        79212968340    6000.0      1
        79213012446    5000.0      1
...                       ...    ...
1310927 79220012822    1100.0      3
        79219057396    5700.0      4
        79215977146    2500.0      5
        79218406274   23000.0      8
        79220081626  155100.0     98

[21902425 rows x 2 columns]

In [14]:
top_3_data = X_data.groupby(level=0).tail(3)

In [15]:
top_3_data

amount  count
payer   phone                       
0       79213906883   18000.0      1
        79216341986     500.0      1
1       79215518554  200000.0      6
        79213034226  207500.0      7
        79213029621  128100.0     15
...                       ...    ...
1310926 79214583333   84500.0     13
        79212557136  124500.0     15
1310927 79215977146    2500.0      5
        79218406274   23000.0      8
        79220081626  155100.0     98

[3726072 rows x 2 columns]

In [22]:
last_person_shaped_data = last_person_data.reset_index(level='date').drop(columns='date')
result_data = top_3_data[['amount']].isin(last_person_shaped_data)

In [23]:
result_data[result_data.amount == True]

,,amount
payer,phone,
25,79215198295,True
74,79216493592,True
216,79211648247,True
223,79214305179,True
285,79216859913,True
...,...,...
1310148,79218964266,True
1310210,79219832610,True
1310442,79219660153,True


In [35]:
print(f'Итоговая точность: {len(result_data[result_data.amount == True])/len(last_person_data)}')

Итоговая точность: 0.014343263441422076
